You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [60]:
import os
from dotenv import load_dotenv
import tiktoken

# Load environment variables from a .env file
load_dotenv()

# Set the OPENAI_API_KEY directly if it's not found in the environment

openai_api_key = os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
#tokenizer = tiktoken.get_encoding("cl100k_base")  # Use the correct encoding for gpt-4
# Explicitly get the tokenizer for gpt-4o
#tokenizer = tiktoken.encoding_for_model("gpt-4o")  # Use the correct encoding for gpt-4

# Use the tokenizer in your agent setup if needed

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [62]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
)



### Agent: Writer

In [61]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [63]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [64]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [65]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [67]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [68]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-05-23 15:31:38,725 - 8021201664 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [69]:
result = crew.kickoff(inputs={"topic": "Dogs and Cats"})

2024-05-23 15:31:58,130 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 15:31:58,131 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 15:31:58,132 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')


 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Dogs and Cats.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...


2024-05-23 15:32:16,719 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 15:32:16,721 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 15:32:16,722 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')


Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.

---

**Comprehensive Content Plan: Dogs and Cats**

### 1. Prioritize the Latest Trends, Key Players, and Noteworthy News on Dogs and Cats

#### Latest Trends:
- **Pet Adoption Rates**: Increased adoption rates during and post-pandemic.
- **Health and Wellness**: Rise in organic pet foods and supplements.
- **Technology**: Growth in pet tech gadgets like tracking collars and smart feeders.
- **Pet-Friendly Policies**: More workplaces and public spaces becoming pet-friendly.
- **Sustainable Pet Products**: Eco-friendly toys, biodegradable waste bags, and sustainable pet food.

#### Key Players:
- **Pet Food Brands**: Blue Buffalo, Hill's Science Diet, Royal Canin.
- **Pet Tech Companies**: Whistle, FitBark, Petcube.
- **Veterinary Services**: Banfield Pet Hospital, VCA Animal Hospitals.
- **Pet Adoption Agencies**: ASPCA, Petfinder, Best Friends Animal Society.

#### Noteworthy News:
- Recen

2024-05-23 15:32:47,864 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 15:32:47,866 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 15:32:47,867 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 15:32:47,867 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_

Thought: I now can give a great answer
Final Answer: 

```markdown
# Dogs and Cats: The Everlasting Companions

Dogs and cats have long been cherished members of our households, providing companionship, joy, and even health benefits. As our lifestyles evolve, so too do the ways we care for our furry friends. This blog post delves into the latest trends, key players, and noteworthy news in the world of dogs and cats, offering valuable insights for pet owners and enthusiasts alike. Whether you're a seasoned pet parent or considering adding a new member to your family, this article aims to equip you with the information you need to make informed decisions.

## Adoption Trends and Tips

### Current Adoption Trends

The global pandemic has seen a significant rise in pet adoption rates, as people sought companionship during lockdowns. According to the ASPCA, there has been a marked increase in both dog and cat adoptions, a trend that continues in the post-pandemic world. This surge in adopti

- Display the results of your execution as markdown in the notebook.

In [70]:
from IPython.display import Markdown
Markdown(result)

my best complete final answer to the task.

```markdown
# Dogs and Cats: The Everlasting Companions

Dogs and cats have long been cherished members of our households, providing companionship, joy, and even health benefits. As our lifestyles evolve, so too do the ways we care for our furry friends. This blog post delves into the latest trends, key players, and noteworthy news in the world of dogs and cats, offering valuable insights for pet owners and enthusiasts alike. Whether you're a seasoned pet parent or considering adding a new member to your family, this article aims to equip you with the information you need to make informed decisions.

## Adoption Trends and Tips

### Current Adoption Trends

The global pandemic has seen a significant rise in pet adoption rates, as people sought companionship during lockdowns. According to the ASPCA, there has been a marked increase in both dog and cat adoptions, a trend that continues in the post-pandemic world. This surge in adoption highlights the enduring appeal of pets and the growing recognition of their benefits to mental health and well-being.

### Pros and Cons of Adopting a Pet

Adopting a pet is a rewarding yet significant commitment. On the plus side, adoption can save a life and reduce the number of animals in shelters. Pets can provide emotional support, companionship, and even health benefits such as reduced stress and increased physical activity. However, potential adopters should also consider the long-term responsibilities, including financial costs, time commitment, and the need for regular veterinary care.

### Choosing the Right Pet for Your Lifestyle

Selecting the right pet involves assessing your lifestyle, living situation, and personal preferences. Dogs, for instance, generally require more time and active engagement, while cats are often more independent. Prospective pet owners should consider factors such as space, activity level, and any existing pets when making their choice. Researching breeds and consulting with adoption agencies can provide valuable guidance in finding the perfect match.

## Health and Wellness

### Importance of Regular Veterinary Care

Regular veterinary care is crucial for the health and longevity of both dogs and cats. Annual check-ups can help detect and prevent common health issues such as dental disease, obesity, and chronic conditions like diabetes and arthritis. Vaccinations, parasite control, and routine blood tests are essential components of a comprehensive pet health plan.

### Organic and Specialized Pet Foods

The rise in organic and specialized pet foods reflects a broader trend towards health and wellness. Many pet owners are opting for high-quality, organic foods free from artificial additives and fillers. Brands like Blue Buffalo, Hill's Science Diet, and Royal Canin offer a range of options tailored to specific dietary needs, including grain-free, hypoallergenic, and age-specific formulas. These specialized diets can support overall health, improve coat condition, and alleviate food sensitivities.

### The Role of Supplements

Supplements are increasingly popular for enhancing pet health. Products containing omega-3 fatty acids, glucosamine, and probiotics can support joint health, improve digestion, and boost the immune system. While supplements can be beneficial, it's essential to consult with a veterinarian to ensure they are appropriate for your pet's specific needs.

## Technology in Pet Care

### Popular Pet Tech Gadgets

Technology is transforming pet care, making it easier to monitor and ensure the well-being of our furry friends. Gadgets like Whistle and FitBark offer advanced tracking collars that monitor activity levels, location, and even health metrics. Petcube provides interactive cameras that allow owners to play with their pets remotely, offering both mental stimulation and peace of mind.

### Benefits of Pet Tech

Pet tech gadgets can significantly enhance pet safety and health monitoring. GPS collars reduce the risk of lost pets, while health trackers can detect early signs of illness, prompting timely veterinary visits. Smart feeders and water fountains ensure pets receive consistent nutrition and hydration, even when owners are away.

### Reviews of Top Pet Tech Products

Top pet tech products have received positive reviews for their innovation and reliability. For example, Whistle's GPS tracker is praised for its accuracy and user-friendly app, while Petcube's interactive camera is lauded for its high-quality video and engaging features. Prospective buyers should consult trusted review sites like PCMag and TechRadar to find the best products for their needs.

## Sustainable Pet Ownership

### Environmental Impact and Mitigation

Pet ownership does have an environmental footprint, but there are ways to reduce it. Choosing eco-friendly products, such as biodegradable waste bags and sustainably sourced pet food, can make a significant difference. Additionally, spaying and neutering pets can help control the population and reduce the strain on shelters and resources.

### Eco-Friendly Pet Products

A growing number of companies are offering eco-friendly pet products. Brands like Earth Rated produce biodegradable waste bags, while others like West Paw create toys from recycled materials. Sustainable pet food options are also available, made from responsibly sourced ingredients and minimal packaging.

### Tips for Sustainable Pet Care

Adopting sustainable practices in pet care is easier than you might think. Simple steps like using reusable grooming tools, choosing eco-friendly toys, and supporting brands committed to sustainability can make a big impact. Additionally, consider DIY pet treats using organic ingredients to reduce packaging waste and ensure quality nutrition.

## Pet Behavior and Training

### Common Behavioral Issues

Behavioral issues are common in pets but can often be addressed with the right approach. Dogs may exhibit problems like excessive barking, chewing, or separation anxiety, while cats might struggle with litter box issues or scratching furniture. Understanding the root cause of these behaviors is the first step towards finding effective solutions.

### Training Tips and Resources

Training is essential for a well-behaved pet and a harmonious household. Positive reinforcement techniques, such as rewarding good behavior with treats or praise, are highly effective. Resources like training classes, books, and online tutorials can provide valuable guidance. Consistency and patience are key to successful training.

### Mental Stimulation and Exercise

Mental stimulation and exercise are vital for your pet's well-being. Regular walks, interactive toys, and puzzle feeders can keep dogs and cats engaged and active. Mental challenges, such as obedience training or agility courses, can also provide much-needed stimulation and prevent boredom-related behaviors.

## Conclusion

Dogs and cats continue to be beloved companions, enriching our lives in countless ways. Staying informed about the latest trends, health tips, and sustainable practices can help ensure that our pets lead happy, healthy lives. Whether you're a current pet owner or considering adoption, the insights provided here aim to support you in making the best choices for your furry friends.

### Call to Action

Subscribe to our blog for more pet care tips and updates. Share your own pet stories and tips in the comments section below. For further reading and trusted resources, explore our curated links to top pet care websites and organizations.

---

By following this comprehensive content plan, we have crafted an engaging and informative blog post that meets the outlined objectives and provides valuable insights for our target audience.
```

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [58]:
topic = "Big Sister AI - AI copilot which helps to evaluate and improve the work of sales reps."
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Big Sister AI - AI copilot which helps to evaluate and improve the work of sales reps..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer.

Final Answer: 

**Content Plan Document for Blog Article: "Big Sister AI: Your Sales Team's New AI Copilot"**

**1. Introduction:**
   - Briefly define Artificial Intelligence (AI) and its evolution in the business sector.
   - Introduce Big Sister AI as an innovative solution designed to assist sales teams in evaluating and improving their performance.

**2. Target Audience:**
   - Sales professionals at all levels, especially those interested in enha

In [59]:
Markdown(result)

```markdown
# Big Sister AI: Your Sales Team's New AI Copilot

## Introduction to Big Sister AI

Welcome to the future of sales performance enhancement! Artificial Intelligence (AI) has been a game-changer in various sectors, and now, it's set to revolutionize the sales industry. Meet Big Sister AI, an innovative AI copilot designed to assist sales teams in evaluating and refining their strategies and outputs. This tool leverages advanced machine learning and data analytics to provide actionable insights, making it an invaluable asset for any sales professional.

AI copilots are not just tools; they are transforming into essential partners across industries. By providing data-driven insights, these AI systems help professionals make informed decisions quickly and accurately. Big Sister AI stands at the forefront of this transformation, tailored to boost the efficiency and effectiveness of sales teams.

## How Big Sister AI Works

What sets Big Sister AI apart? At its core, this AI utilizes cutting-edge machine learning algorithms and comprehensive data analytics to observe and analyze sales interactions and outcomes. The system learns from each interaction, offering tailored feedback to sales representatives.

Consider a hypothetical scenario where a sales rep struggles with closing deals. Big Sister AI can analyze past interactions, compare them with more successful cases, and suggest actionable adjustments or training modules. This real-time feedback allows sales reps to adapt and enhance their techniques on the fly.

## Benefits of Implementing Big Sister AI

The implementation of Big Sister AI into your sales strategy can significantly elevate the team's performance. One of the primary benefits is the provision of real-time feedback and analytics. This immediate insight helps sales reps understand which strategies are working and which need adjustment, fostering a continuous improvement environment.

Moreover, Big Sister AI plays a crucial role in personalized coaching and development for sales reps. Each member of the team receives tailored feedback based on their interactions, which helps in developing their skills in a way that's specifically beneficial to their style and needs, ultimately boosting overall team performance.

## Challenges and Considerations

While the benefits are substantial, there are challenges and considerations to acknowledge when integrating Big Sister AI. The privacy of data is a paramount concern; ensuring that all personal and customer data handled by AI is protected is essential. Additionally, teams may face a learning curve when adopting new technologies. It's crucial to provide adequate training and support during the integration phase.

To integrate Big Sister AI seamlessly, start with pilot programs to acclimate the team to the new system and gather feedback on its functionality and usefulness, making adjustments before a full rollout.

## Key Players and Latest Trends

The landscape of AI in sales is dynamic, with several key players leading the charge. Companies like Salesforce and HubSpot are integrating AI tools similar to Big Sister AI, enhancing their platforms' capabilities. Staying updated with these advancements can provide strategic insights and competitive advantages in leveraging AI effectively.

Recent trends indicate a surge in AI adoption across sales teams, driven by the need for more robust data-driven decision-making and efficiency in sales processes. As these technologies evolve, they are set to become even more sophisticated and integral to sales strategies.

## Future of AI in Sales

Looking ahead, the role of AI in sales is not just promising; it's pivotal. As AI technologies continue to evolve, their capacity to analyze complex datasets and provide deeper insights will only grow, making them indispensable tools for sales teams aiming to stay ahead of the curve.

We'd love to hear your thoughts on the integration of AI like Big Sister AI in sales practices. Do you see it as a revolutionary step, or are there concerns you feel need addressing?

## Call to Action

Are you ready to catapult your sales team into a new era of efficiency and success? Consider how Big Sister AI can redefine your approach to sales management and strategy. For more insights and a deeper dive into AI applications in sales, check out our additional resources and product demos available on our website.

Explore, engage, and elevate your sales team with the power of AI!
```
This comprehensive blog post provides an insightful overview of Big Sister AI and its impact on sales teams, aligning with the content plan and integrating SEO keywords to enhance visibility and engagement.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).